In [1]:
import numpy as np
import tensorflow as tf
from ulit import *
from network import *

In [2]:
# 训练神经网络
def train():
    notes = get_notes()

    # 得到所有不重复（因为用了 set）的音调数目
    num_pitch = len(set(notes))

    network_input, network_output = prepare_sequences(notes, num_pitch)

    model = network_model(network_input, num_pitch)

    filepath = "weights-{epoch:02d}-{loss:.4f}.hdf5"

    # 用 Checkpoint（检查点）文件在每一个 Epoch 结束时保存模型的参数（Weights）
    # 不怕训练过程中丢失模型参数。可以在我们对 Loss（损失）满意了的时候随时停止训练
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath,  # 保存的文件路径
        monitor='loss',  # 监控的对象是 损失（loss）
        verbose=0,
        save_best_only=True,  # 不替换最近的数值最佳的监控对象的文件
        mode='min'      # 取损失最小的
    )
    callbacks_list = [checkpoint]

    # 用 fit 方法来训练模型
    model.fit(network_input, network_output, epochs=1, batch_size=64, callbacks=callbacks_list)


In [3]:
def prepare_sequences(notes, num_pitch):
    """
    为神经网络准备好供训练的序列
    """
    sequence_length = 100  # 序列长度

    # 得到所有音调的名字
    pitch_names = sorted(set(item for item in notes))

    # 创建一个字典，用于映射 音调 和 整数
    pitch_to_int = dict((pitch, num) for num, pitch in enumerate(pitch_names))

    # 创建神经网络的输入序列和输出序列
    network_input = [] #输入序列
    network_output = [] #标签

    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i: i + sequence_length] #输入
        sequence_out = notes[i + sequence_length] #真实值

        network_input.append([pitch_to_int[char] for char in sequence_in]) #通过字典将音调转化为数
        network_output.append(pitch_to_int[sequence_out])

    n_patterns = len(network_input)

    # 将输入的形状转换成神经网络模型可以接受的
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

    # 将 输入 标准化 / 归一化
    # 归一话可以让之后的优化器（optimizer）更快更好地找到误差最小值
    network_input = network_input / float(num_pitch)

    # 将期望输出转换成 {0, 1} 组成的布尔矩阵，为了配合 categorical_crossentropy 误差算法使用
    network_output = tf.keras.utils.to_categorical(network_output)

    return network_input, network_output

In [5]:
if __name__ == '__main__':
    train()

Epoch 1/1
  128/42685 [..............................]  128/42685 [..............................] - ETA: 31:01 - loss: 5.7827

KeyboardInterrupt: 

['E5', 'E5', 'F#5', 'E5', 'D3', 'A3', 'A4', 'C#4', 'C#5', 'F#4', 'D5', 'B4', 'D3', 'G#3', 'E5', 'B3', 'E4', 'E5', 'F#5', 'E5', 'C#3', 'G#3', 'E4', 'B3', 'F#4', 'C#4', 'G#4', 'A4', '4.6', 'C#3', 'E5', 'G#3', '4.9', '4.9', 'F#4', 'E5', 'A4', 'B2', 'F#3', 'D5', 'C#4', 'D5', 'D4', 'A4', 'F4', 'C#5', 'E3', 'B3', 'B4', 'D4', 'A4', 'A4', '2.4', 'A2', 'E3', 'A3', 'D4', '11.1', 'A2', 'A2', 'A4', 'B4', '1.4', '9.1', 'A4', '11.3.6', 'A2', 'A4', 'F#4', '3.9', 'B4', '2.5', 'A2', 'A4', 'A4', '9.2', 'F4', '1.4', 'A2', 'C#5', 'C#5', 'G#2', 'E4', 'F#2', 'A4', 'B4', '1.6', '6.9.1', 'A4', '11.3.6', '9.11', 'C#5', 'F#4', '3.9', 'F#4', 'G#4', '9.2', 'E2', 'A4', 'A4', '9.2', 'B4', '8.11.2', 'E2', '2.8', 'E3', 'A2', 'A4', 'E3', 'B4', '1.4', '9.1', 'A4', '11.3.6', 'A2', 'A4', 'F#3', 'F#4', '3.9', 'B4', '2.5', 'A2', 'A4', 'F3', 'A4', '9.2', 'F4', '1.4', 'A2', 'C#5', 'E3', 'C#5', '9.1', 'G#2', 'F#2', 'A4', 'C#3', 'B4', '1.6', '6.9', 'A4', '11.3.6', 'B2', 'A4', 'F#3', 'B4', 'A3', 'F#4', 'E2', 'E4', 'E3', 'E4', '

40296